In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pylab as plt 
import quantities as pq
from bsb import Scaffold, parse_configuration_file
from IPython.display import HTML, display
from utils import extract_device_data, plot_signal

In [ ]:
# bsb.options.verbosity = 3
config = parse_configuration_file("configurations/dcn_io.yaml", parser="yaml")

scaffold = Scaffold(config)
scaffold.compile(clear=True)

In [ ]:
storage = scaffold.storage

for cell_name, cell_type in scaffold.cell_types.items():
    ps = cell_type.get_placement_set() 
    pos = ps.load_positions()
    print(f"{len(pos)} {cell_name} placed")

In [ ]:
cs = scaffold.get_connectivity_set("dcn_i_to_io")
from_DCNi_to_IO = {}
to_IO_from_DCNi = {}
for pre_locs, post_locs in cs.load_connections():
    pre_cell_id = pre_locs[0]
    post_cell_id = post_locs[0]

    if pre_cell_id in from_DCNi_to_IO:
        from_DCNi_to_IO[pre_cell_id].append(post_cell_id)
    else:
        from_DCNi_to_IO[pre_cell_id] = [post_cell_id]

    if post_cell_id in to_IO_from_DCNi:
        to_IO_from_DCNi[post_cell_id].append(pre_cell_id)
    else:
        to_IO_from_DCNi[post_cell_id] = [pre_cell_id]

In [ ]:
results = scaffold.run_simulation("basal_activity")
spiketrains = results.spiketrains
signals = results.analogsignals

In [ ]:
# Extract device data
devices_data = extract_device_data(config)

# Create a DataFrame for better display
devices_df = pd.DataFrame(devices_data)
HTML(devices_df.to_html(index=False))


In [ ]:
plot_signal(signal=signals[0], devices=devices_data, title="IO EGLIF Membrane Potential", output="test.png")